# Bid-Ask Order Book Analysis

Visualize order book depth from both eBay and Blokpax data.

**Data Sources:**
- **eBay ASK**: `marketprice` table where `listing_type = 'active'`
- **eBay BID**: `highest_bid` from `marketsnapshot` (limited - no depth)
- **Blokpax ASK**: `blokpaxlisting` table where `status = 'active'`
- **Blokpax BID**: `blokpaxoffer` table where `status = 'open'`

In [ ]:
import sys
sys.path.insert(0, '..')

from datetime import datetime, timedelta, timezone
from sqlalchemy import text
from app.db import engine
import numpy as np

## 1. Query Functions

In [ ]:
def get_ebay_asks(card_id: int, days: int = 30) -> list[dict]:
    """Get eBay active listings (ASK side)."""
    cutoff = datetime.now(timezone.utc) - timedelta(days=days)
    query = text("""
        SELECT price, treatment, title, scraped_at
        FROM marketprice
        WHERE card_id = :card_id
          AND listing_type = 'active'
          AND scraped_at >= :cutoff
          AND is_bulk_lot = FALSE
          AND platform = 'ebay'
        ORDER BY price ASC
    """)
    with engine.connect() as conn:
        result = conn.execute(query, {"card_id": card_id, "cutoff": cutoff})
        return [dict(row._mapping) for row in result.fetchall()]


def get_ebay_highest_bid(card_id: int) -> float | None:
    """Get eBay highest bid from snapshots (limited BID data)."""
    query = text("""
        SELECT highest_bid
        FROM marketsnapshot
        WHERE card_id = :card_id
          AND highest_bid IS NOT NULL
        ORDER BY timestamp DESC
        LIMIT 1
    """)
    with engine.connect() as conn:
        result = conn.execute(query, {"card_id": card_id}).fetchone()
        return float(result[0]) if result else None


def get_blokpax_asks(asset_id: str) -> list[dict]:
    """Get Blokpax active listings (ASK side)."""
    query = text("""
        SELECT price_usd as price, seller_address, quantity, created_at
        FROM blokpaxlisting
        WHERE asset_id = :asset_id
          AND status = 'active'
        ORDER BY price_usd ASC
    """)
    with engine.connect() as conn:
        result = conn.execute(query, {"asset_id": asset_id})
        return [dict(row._mapping) for row in result.fetchall()]


def get_blokpax_bids(asset_id: str) -> list[dict]:
    """Get Blokpax open offers (BID side)."""
    query = text("""
        SELECT price_usd as price, buyer_address, quantity, created_at
        FROM blokpaxoffer
        WHERE asset_id = :asset_id
          AND status = 'open'
        ORDER BY price_usd DESC
    """)
    with engine.connect() as conn:
        result = conn.execute(query, {"asset_id": asset_id})
        return [dict(row._mapping) for row in result.fetchall()]

## 2. ASCII Order Book Visualization

In [ ]:
def create_price_buckets(prices: list[float], num_buckets: int = 10) -> list[dict]:
    """Create price buckets for visualization."""
    if not prices:
        return []
    
    min_p, max_p = min(prices), max(prices)
    if min_p == max_p:
        return [{"min": min_p, "max": max_p, "count": len(prices), "mid": min_p}]
    
    bucket_width = (max_p - min_p) / num_buckets
    buckets = []
    
    for i in range(num_buckets):
        bucket_min = min_p + i * bucket_width
        bucket_max = min_p + (i + 1) * bucket_width
        count = sum(1 for p in prices if bucket_min <= p < bucket_max)
        if i == num_buckets - 1:  # Include max in last bucket
            count = sum(1 for p in prices if bucket_min <= p <= bucket_max)
        buckets.append({
            "min": round(bucket_min, 2),
            "max": round(bucket_max, 2),
            "mid": round((bucket_min + bucket_max) / 2, 2),
            "count": count
        })
    
    return buckets


def render_ascii_order_book(
    bids: list[dict],
    asks: list[dict],
    title: str = "Order Book",
    width: int = 60,
    num_buckets: int = 8
) -> str:
    """Render ASCII order book with bids on left, asks on right."""
    bid_prices = [b["price"] for b in bids if b.get("price")]
    ask_prices = [a["price"] for a in asks if a.get("price")]
    
    bid_buckets = create_price_buckets(bid_prices, num_buckets)
    ask_buckets = create_price_buckets(ask_prices, num_buckets)
    
    # Find max count for scaling bars
    max_count = max(
        max((b["count"] for b in bid_buckets), default=0),
        max((b["count"] for b in ask_buckets), default=0),
        1
    )
    
    bar_width = (width - 20) // 2  # Space for price labels
    
    lines = []
    lines.append(f"\n{'=' * width}")
    lines.append(f"{title:^{width}}")
    lines.append(f"{'=' * width}")
    
    # Stats
    best_bid = max(bid_prices) if bid_prices else 0
    best_ask = min(ask_prices) if ask_prices else 0
    spread = best_ask - best_bid if best_bid and best_ask else 0
    spread_pct = (spread / best_ask * 100) if best_ask else 0
    
    lines.append(f"  Best Bid: ${best_bid:>8.2f}  |  Best Ask: ${best_ask:<8.2f}")
    lines.append(f"  Spread: ${spread:>8.2f} ({spread_pct:.1f}%)")
    lines.append(f"  Bid Depth: {len(bid_prices):>5}     |  Ask Depth: {len(ask_prices):<5}")
    lines.append(f"{'-' * width}")
    lines.append(f"{'BID (Buyers)':^{width//2}}|{'ASK (Sellers)':^{width//2}}")
    lines.append(f"{'-' * width}")
    
    # Combine and align buckets around the spread
    # Bids: highest at top (descending)
    # Asks: lowest at top (ascending)
    bid_buckets_rev = list(reversed(bid_buckets))  # Highest first
    
    max_rows = max(len(bid_buckets_rev), len(ask_buckets))
    
    for i in range(max_rows):
        # Bid side (left)
        if i < len(bid_buckets_rev):
            b = bid_buckets_rev[i]
            bar_len = int(b["count"] / max_count * bar_width)
            bid_bar = '█' * bar_len
            bid_str = f"${b['mid']:>6.2f} {bid_bar:>{bar_width}} {b['count']:>3}"
        else:
            bid_str = " " * (width // 2 - 1)
        
        # Ask side (right)
        if i < len(ask_buckets):
            a = ask_buckets[i]
            bar_len = int(a["count"] / max_count * bar_width)
            ask_bar = '█' * bar_len
            ask_str = f"{a['count']:>3} {ask_bar:<{bar_width}} ${a['mid']:<6.2f}"
        else:
            ask_str = " " * (width // 2 - 1)
        
        lines.append(f"{bid_str}|{ask_str}")
    
    lines.append(f"{'=' * width}\n")
    
    return "\n".join(lines)

## 3. Combined Order Book (eBay + Blokpax)

In [ ]:
def get_combined_order_book(
    card_id: int,
    blokpax_asset_id: str | None = None,
    days: int = 30
) -> dict:
    """
    Get combined order book from eBay and Blokpax.
    
    Returns:
        {
            "asks": [...],  # All ask-side listings
            "bids": [...],  # All bid-side offers
            "ebay_highest_bid": float | None,
            "sources": {"ebay_asks": n, "blokpax_asks": n, "blokpax_bids": n}
        }
    """
    asks = []
    bids = []
    sources = {"ebay_asks": 0, "blokpax_asks": 0, "blokpax_bids": 0}
    
    # eBay ASK side
    ebay_asks = get_ebay_asks(card_id, days)
    for a in ebay_asks:
        a["source"] = "ebay"
    asks.extend(ebay_asks)
    sources["ebay_asks"] = len(ebay_asks)
    
    # eBay BID side (limited - just highest bid)
    ebay_highest_bid = get_ebay_highest_bid(card_id)
    
    # Blokpax data if asset_id provided
    if blokpax_asset_id:
        blokpax_asks = get_blokpax_asks(blokpax_asset_id)
        for a in blokpax_asks:
            a["source"] = "blokpax"
        asks.extend(blokpax_asks)
        sources["blokpax_asks"] = len(blokpax_asks)
        
        blokpax_bids = get_blokpax_bids(blokpax_asset_id)
        for b in blokpax_bids:
            b["source"] = "blokpax"
        bids.extend(blokpax_bids)
        sources["blokpax_bids"] = len(blokpax_bids)
    
    # Sort
    asks.sort(key=lambda x: x["price"])
    bids.sort(key=lambda x: x["price"], reverse=True)
    
    return {
        "asks": asks,
        "bids": bids,
        "ebay_highest_bid": ebay_highest_bid,
        "sources": sources
    }

## 4. Test with Sample Card

In [ ]:
# Find a card with active listings
query = text("""
    SELECT c.id, c.name, COUNT(*) as listing_count
    FROM card c
    JOIN marketprice mp ON mp.card_id = c.id
    WHERE mp.listing_type = 'active'
      AND mp.scraped_at >= NOW() - INTERVAL '30 days'
    GROUP BY c.id, c.name
    ORDER BY listing_count DESC
    LIMIT 10
""")

with engine.connect() as conn:
    result = conn.execute(query)
    top_cards = [dict(row._mapping) for row in result.fetchall()]

print("Top cards by active listing count:")
for card in top_cards:
    print(f"  [{card['id']:>3}] {card['name']:<40} ({card['listing_count']} listings)")

In [ ]:
# Pick a card to analyze (update this ID based on above results)
CARD_ID = top_cards[0]["id"] if top_cards else 1
CARD_NAME = top_cards[0]["name"] if top_cards else "Unknown"

print(f"Analyzing: {CARD_NAME} (ID: {CARD_ID})")

In [ ]:
# Get eBay order book (ASK side only - no bid depth)
ebay_asks = get_ebay_asks(CARD_ID)
ebay_highest_bid = get_ebay_highest_bid(CARD_ID)

print(f"eBay Data for {CARD_NAME}:")
print(f"  ASK side: {len(ebay_asks)} active listings")
print(f"  BID side: highest_bid = ${ebay_highest_bid:.2f}" if ebay_highest_bid else "  BID side: No bid data")

if ebay_asks:
    print(f"\n  Price range: ${min(a['price'] for a in ebay_asks):.2f} - ${max(a['price'] for a in ebay_asks):.2f}")
    print(f"  Lowest 5 asks:")
    for a in ebay_asks[:5]:
        print(f"    ${a['price']:>8.2f} - {a.get('treatment', 'N/A'):<20}")

In [ ]:
# Visualize eBay order book (ASK only, with highest bid marker)
# Since we don't have bid depth, we'll show a single bid point

bids_for_viz = []
if ebay_highest_bid:
    # Create synthetic "bid" entries to show highest bid level
    bids_for_viz = [{"price": ebay_highest_bid, "source": "ebay"}]

print(render_ascii_order_book(
    bids=bids_for_viz,
    asks=ebay_asks,
    title=f"eBay Order Book: {CARD_NAME}",
    width=70
))

if not ebay_highest_bid:
    print("Note: eBay bid depth not available (only highest_bid tracked)")

## 5. Blokpax Order Book (Full Bid-Ask)

In [ ]:
# Find Blokpax assets with both listings and offers
query = text("""
    SELECT 
        a.external_id,
        a.name,
        a.card_id,
        (SELECT COUNT(*) FROM blokpaxlisting l WHERE l.asset_id = a.external_id AND l.status = 'active') as listing_count,
        (SELECT COUNT(*) FROM blokpaxoffer o WHERE o.asset_id = a.external_id AND o.status = 'open') as offer_count
    FROM blokpax_asset a
    WHERE a.card_id IS NOT NULL
    ORDER BY (SELECT COUNT(*) FROM blokpaxoffer o WHERE o.asset_id = a.external_id AND o.status = 'open') DESC
    LIMIT 10
""")

with engine.connect() as conn:
    result = conn.execute(query)
    blokpax_assets = [dict(row._mapping) for row in result.fetchall()]

print("Blokpax assets with offer activity:")
for asset in blokpax_assets:
    print(f"  {asset['name']:<40} L:{asset['listing_count']:>3} O:{asset['offer_count']:>3}")

In [ ]:
# Pick a Blokpax asset with offers
if blokpax_assets and blokpax_assets[0]["offer_count"] > 0:
    BPX_ASSET = blokpax_assets[0]
    BPX_ASSET_ID = BPX_ASSET["external_id"]
    BPX_ASSET_NAME = BPX_ASSET["name"]
    
    bpx_asks = get_blokpax_asks(BPX_ASSET_ID)
    bpx_bids = get_blokpax_bids(BPX_ASSET_ID)
    
    print(f"Blokpax Data for {BPX_ASSET_NAME}:")
    print(f"  ASK side: {len(bpx_asks)} active listings")
    print(f"  BID side: {len(bpx_bids)} open offers")
    
    print(render_ascii_order_book(
        bids=bpx_bids,
        asks=bpx_asks,
        title=f"Blokpax Order Book: {BPX_ASSET_NAME}",
        width=70
    ))
else:
    print("No Blokpax assets with active offers found.")

## 6. Depth Chart Visualization

In [ ]:
def render_depth_chart(bids: list[dict], asks: list[dict], width: int = 70) -> str:
    """
    Render cumulative depth chart showing liquidity at each price level.
    
    BID side: cumulative from highest to lowest
    ASK side: cumulative from lowest to highest
    """
    bid_prices = sorted([b["price"] for b in bids if b.get("price")], reverse=True)
    ask_prices = sorted([a["price"] for a in asks if a.get("price")])
    
    if not bid_prices and not ask_prices:
        return "No data for depth chart"
    
    lines = []
    lines.append(f"\n{'=' * width}")
    lines.append(f"{'DEPTH CHART':^{width}}")
    lines.append(f"{'=' * width}")
    
    # Calculate cumulative depths
    bid_depth = []
    cumulative = 0
    for p in bid_prices:
        cumulative += 1
        bid_depth.append((p, cumulative))
    
    ask_depth = []
    cumulative = 0
    for p in ask_prices:
        cumulative += 1
        ask_depth.append((p, cumulative))
    
    max_depth = max(
        max((d[1] for d in bid_depth), default=0),
        max((d[1] for d in ask_depth), default=0),
        1
    )
    
    # Create price levels for visualization
    all_prices = bid_prices + ask_prices
    if not all_prices:
        return "No price data"
    
    min_p, max_p = min(all_prices), max(all_prices)
    price_range = max_p - min_p
    if price_range == 0:
        price_range = 1
    
    num_levels = 15
    bar_width = width - 25
    
    lines.append(f"{'Price':>10} {'BID Depth':^{bar_width//2}}|{'ASK Depth':^{bar_width//2}}")
    lines.append(f"{'-' * width}")
    
    for i in range(num_levels):
        price_level = max_p - (i * price_range / (num_levels - 1))
        
        # Find cumulative bid depth at this price (bids >= price_level)
        bid_cum = sum(1 for p in bid_prices if p >= price_level)
        
        # Find cumulative ask depth at this price (asks <= price_level)
        ask_cum = sum(1 for p in ask_prices if p <= price_level)
        
        bid_bar_len = int(bid_cum / max_depth * (bar_width // 2 - 2))
        ask_bar_len = int(ask_cum / max_depth * (bar_width // 2 - 2))
        
        bid_bar = '█' * bid_bar_len
        ask_bar = '█' * ask_bar_len
        
        lines.append(f"${price_level:>8.2f} {bid_bar:>{bar_width//2-1}}|{ask_bar:<{bar_width//2-1}}")
    
    lines.append(f"{'=' * width}")
    lines.append(f"  Total BID depth: {len(bid_prices):>5}  |  Total ASK depth: {len(ask_prices):<5}")
    lines.append("")
    
    return "\n".join(lines)


# Show depth chart for eBay
print(render_depth_chart(
    bids=bids_for_viz,
    asks=ebay_asks
))

## 7. Bid-Ask Spread Analysis

In [ ]:
def analyze_spread(bids: list[dict], asks: list[dict], highest_bid_override: float | None = None) -> dict:
    """
    Analyze bid-ask spread and market efficiency.
    """
    bid_prices = [b["price"] for b in bids if b.get("price")]
    ask_prices = [a["price"] for a in asks if a.get("price")]
    
    best_bid = max(bid_prices) if bid_prices else highest_bid_override
    best_ask = min(ask_prices) if ask_prices else None
    
    if not best_bid or not best_ask:
        return {"error": "Insufficient data for spread analysis"}
    
    spread = best_ask - best_bid
    spread_pct = (spread / best_ask) * 100
    mid_price = (best_bid + best_ask) / 2
    
    # Market efficiency indicators
    # Tight spread (<5%) = efficient market
    # Wide spread (>20%) = illiquid market
    if spread_pct < 5:
        efficiency = "HIGH (tight spread)"
    elif spread_pct < 10:
        efficiency = "MEDIUM"
    elif spread_pct < 20:
        efficiency = "LOW (wide spread)"
    else:
        efficiency = "VERY LOW (illiquid)"
    
    return {
        "best_bid": best_bid,
        "best_ask": best_ask,
        "spread": spread,
        "spread_pct": spread_pct,
        "mid_price": mid_price,
        "bid_depth": len(bid_prices),
        "ask_depth": len(ask_prices),
        "efficiency": efficiency
    }


# Analyze eBay spread
ebay_spread = analyze_spread(bids_for_viz, ebay_asks, ebay_highest_bid)
print(f"eBay Spread Analysis for {CARD_NAME}:")
if "error" not in ebay_spread:
    print(f"  Best Bid:    ${ebay_spread['best_bid']:>8.2f}")
    print(f"  Best Ask:    ${ebay_spread['best_ask']:>8.2f}")
    print(f"  Spread:      ${ebay_spread['spread']:>8.2f} ({ebay_spread['spread_pct']:.1f}%)")
    print(f"  Mid Price:   ${ebay_spread['mid_price']:>8.2f}")
    print(f"  Efficiency:  {ebay_spread['efficiency']}")
else:
    print(f"  {ebay_spread['error']}")

## 8. Next Steps for OrderBookAnalyzer

Based on this analysis, we should update `OrderBookAnalyzer` to:

1. **Include bid data** when available (Blokpax offers)
2. **Calculate true spread** instead of just ask-side floor
3. **Return mid-price** as a more accurate floor estimate when bid data exists
4. **Add spread-based confidence** - tighter spread = higher confidence

In [ ]:
# Summary of data availability
print("\n" + "=" * 60)
print("DATA AVAILABILITY SUMMARY")
print("=" * 60)
print("\neBay:")
print("  ASK: Full depth (marketprice.listing_type='active')")
print("  BID: Limited (marketsnapshot.highest_bid only)")
print("\nBlokpax:")
print("  ASK: Full depth (blokpaxlisting.status='active')")
print("  BID: Full depth (blokpaxoffer.status='open')")
print("\nRecommendation:")
print("  - Use Blokpax for true bid-ask spread analysis")
print("  - Use eBay asks + Blokpax bids for cross-platform view")
print("  - Fall back to ask-only floor when no bid data")
print("=" * 60)